In [ ]:
!pip install openai
!pip install anthropic

import openai
import anthropic
from getpass import getpass

class AnsiColors:
    USER = '\033[96m'
    ASSISTANT = '\033[92m'
    ERROR = '\033[91m'
    ENDC = '\033[0m'

# Constants
OPENAI_MODEL = "gpt-3.5-turbo"
ANTHROPIC_MODEL = "claude-v1"
PROMPTS = ["How would you assess the following proposal and what overall edits would you suggest, and how could the proposal be improved to more successfully meet the stated goal?", "In view of the following assessment and suggested improvements for your proposal, how, if at all, would you amend the proposal to give it the best opportunity to achieve the goal? Please include bullet points for each change you would include and bullet points for each update you would reject, including the reasons for each", "Please use the following feedback to redraft the proposal, and add a table in markdown at the end of the proposal enumerating each key way the proposal is designed to achieve the goal."]
TEMPERATURE = 0.8
MAX_TOKENS = 500

# Get API keys
openai_api_key = getpass('Enter your OpenAI API key: ')
anthropic_api_key = getpass('Enter your Anthropic API key: ')

# Initialize
openai.api_key = openai_api_key
anthropic_client = anthropic.Client(anthropic_api_key)
messages = []

# Get user input
user_input = input(AnsiColors.USER + "Enter your prompt: " + AnsiColors.ENDC)

# 1) Send user prompt to ChatGPT through OpenAI API
messages.append({"role": "user", "content": user_input})

response = openai.ChatCompletion.create(
    model=OPENAI_MODEL,
    messages=messages,
    temperature=TEMPERATURE
)

response_text = response['choices'][0]['message']['content']
print(AnsiColors.ASSISTANT + "OpenAI ChatGPT: " + response_text + AnsiColors.ENDC)

markdown_transcript = f"## Transcript\n\n## Initial User Prompt\n{user_input}\n\n## ChatGPT Response\n{response_text}\n\n"

for idx, prompt in enumerate(PROMPTS):
    if idx % 2 == 0:
        # Even indices: ChatGPT
        messages.append({"role": "user", "content": prompt + " " + response_text})
        response = openai.ChatCompletion.create(
            model=OPENAI_MODEL,
            messages=messages,
            temperature=TEMPERATURE
        )
        response_text = response['choices'][0]['message']['content']
        print(AnsiColors.ASSISTANT + "OpenAI ChatGPT: " + response_text + AnsiColors.ENDC)
        markdown_transcript += f"## ChatGPT Prompt\n{prompt}\n\n## ChatGPT Response\n{response_text}\n\n"
    else:
        # Odd indices: Claude
        anthropic_prompt = "\n\nHuman:" + prompt + " " + response_text + "\n\nAssistant:"
        anthropic_response = anthropic_client.completion(
            prompt=anthropic_prompt,
            model=ANTHROPIC_MODEL,
            temperature=TEMPERATURE,
            max_tokens_to_sample=MAX_TOKENS
        )
        response_text = anthropic_response["completion"]
        print(AnsiColors.ASSISTANT + "Anthropic Claude: " + response_text + AnsiColors.ENDC)
        markdown_transcript += f"## Claude Prompt\n{prompt}\n\n## Claude Response\n{response_text}\n\n"

markdown_transcript += "\n\n---\n\n"

print("\n\n" + AnsiColors.ASSISTANT + "Transcript in Markdown Format:" + AnsiColors.ENDC)
print(markdown_transcript)